In [6]:

import torch
from torch import nn
import pickle

In [7]:
with open('./.temp/imdb.pkl', 'rb') as f:
    imdb_tensor = pickle.load(f)
with open('./.temp/labels.pkl', 'rb') as f:
    imdb_label = pickle.load(f)
with open('./.temp/em.pkl', 'rb') as f:
    embedding_matrix = pickle.load(f)

In [8]:
# model
class CNN(nn.Module):
    def __init__(self, embedding_matrix, dim_in, dim_conv, dim_out, dropout_rate=0.5, max_length=0):
        super(CNN, self).__init__()
        self.embedding = nn.Embedding(embedding_matrix.shape[0], dim_in).from_pretrained(embedding_matrix, freeze=True)
        self.conv1_1 = nn.Conv1d(dim_in, dim_conv, 3, padding=5)
        self.conv1_2 = nn.Conv1d(dim_in, dim_conv, 4, padding=5)
        self.conv1_3 = nn.Conv1d(dim_in, dim_conv, 5, padding=5)
        self.conv1_4 = nn.Conv1d(dim_in, dim_conv, 6, padding=5)
        self.ReLU = nn.ReLU()
        self.maxpool_1 = nn.MaxPool1d(kernel_size=max_length+8)
        self.maxpool_2 = nn.MaxPool1d(kernel_size=max_length+7)
        self.maxpool_3 = nn.MaxPool1d(kernel_size=max_length+6)
        self.maxpool_4 = nn.MaxPool1d(kernel_size=max_length+5)
        self.fc_1 = nn.Linear(dim_conv * 4, dim_out)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.softmax = nn.Softmax()


    def forward(self, x):
        x = self.embedding(x).transpose(1, 2)
        x1 = self.conv1_1(x)
        x1 = self.ReLU(x1)
        x1 = self.maxpool_1(x1)


        x2 = self.conv1_2(x)
        x2 = self.ReLU(x2)
        x2 = self.maxpool_2(x2)

        x3 = self.conv1_3(x)
        x3 = self.ReLU(x3)
        x3 = self.maxpool_3(x3)

        x4 = self.conv1_4(x)
        x4 = self.ReLU(x4)
        x4 = self.maxpool_4(x4)

        x = torch.cat((x1, x2, x3, x4), dim=1)
        x = torch.flatten(x, 1)
        x = self.fc_1(x)
        x = self.dropout(x)
        x = self.softmax(x)
        return x

In [9]:
dim_in = 300
dim_conv = 256
dim_out = 2
lr = 0.001

In [10]:
max_length_2 = 1506
model = CNN(embedding_matrix, dim_in, dim_conv,
            dim_out, max_length=max_length_2)


model.load_state_dict(torch.load("./model/CNN_vector_model.pth"))

model.eval()

acc = 0

for i in range(50):
    pred = model(imdb_tensor[1000 * i : 1000 * (i + 1)])
    label = imdb_label[1000 * i : 1000 * (i + 1)]
    correct_num = (torch.eq(torch.argmax(pred, dim=1), torch.tensor(label))).type(torch.float).sum().item()
    test_acc = correct_num / 1000
    acc += test_acc
print(f"Accuracy: {acc / 50}")

e:\git\SC4001\Project\venv\lib\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Accuracy: 0.8383400000000001
